# Apply Random Forest

In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(e1071)
library(dplyr)
library(randomForest)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'TimeHeadway')
# invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150')

In [4]:
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [5]:
printf <- function(...) cat(sprintf(...))

In [6]:
CV <- function(dfx, verbose=FALSE) {        
    folds <- createFolds(dfx$flag)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    shorts <- c()
    for (ids in folds) {
        train <- dfx[-ids, c(valuables, "flag")]
        test <- dfx[ids, ]
        fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=500)
        p <- predict(fit, newdata=test)
        
        if (verbose) {            
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, p)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            shorts <- c(shorts, shortErr(test, p))
            count <- count + 1
            printf("\n")
        }
        
        nerr <- sum(p != test$flag)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(shorts), mean(reds), mean(blues)))
}

In [21]:
shortErr <- function(test, predictedFlags) {
    predictedRedRows <- test[predictedFlags == "Red", ]
    nRedAccel <- nrow(filter(predictedRedRows, flag == "Red" & AccelOrBrake == "Accel"))
    nBlueAccel <- nrow(filter(predictedRedRows, flag == "Blue" & AccelOrBrake == "Accel"))
    nPredictedRedsAB <- nRedAccel + nBlueAccel
    printf("Short: accel/predictRedAB = %d/%d = %f\n", nRedAccel, nPredictedRedsAB, nRedAccel / nPredictedRedsAB)
    print(predictedRedRows %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    nRedAccel / nPredictedRedsAB
}

In [8]:
correctVsPredict <- function(test, predictedFlags) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum(predictedRedRows$flag == 'Red')
    nPredictedReds <- nrow(predictedRedRows)
    printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum(predictedBlueRows$flag == 'Blue')
    nPredictedBlues <- nrow(predictedBlueRows)
    printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [9]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [10]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [11]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [12]:
df3 %>% group_by(flag) %>% summarize(count=n())

flag,count
BlueA,77
BlueB,372
RedA,401
RedB,192


In [13]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [14]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [15]:
fomula <- paste("flag ~ ", paste(valuables, collapse=" + "))

In [16]:
set.seed(1)

In [17]:
df <- df3[, c(valuables, "AccelOrBrake", "flag")]

In [18]:
result <- CV(df, verbose=TRUE)
printRedRatios(df)
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2])
printf("CV value: %f", result[1])

Fold1
As for Red: correct/predict = 42/58 = 0.724138
As for Blue: correct/predict = 28/46 = 0.608696
Accel: accel/predictRedAB = 29/33 = 0.878788
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     4
2   Blue        Brake    12
3    Red        Accel    29
4    Red        Brake    13

Fold2
As for Red: correct/predict = 48/70 = 0.685714
As for Blue: correct/predict = 23/35 = 0.657143
Accel: accel/predictRedAB = 37/42 = 0.880952
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     5
2   Blue        Brake    17
3    Red        Accel    37
4    Red        Brake    11

Fold3
As for Red: correct/predict = 48/77 = 0.623377
As for Blue: correct/predict = 16/27 = 0.592593
Accel: accel/predictRedAB = 29/34 = 0.852941
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue       

In [19]:
set.seed(1)

In [20]:
# ntree.lim <- seq(50, 600, 30)

tune <- tune.randomForest(
        flag ~ .,
        data=dfx,
        ntree=ntree.lim,
        tunecontrol = tune.control(sampling = "cross", cross = 10)
)

ERROR: Error in tune.randomForest(flag ~ ., data = dfx, ntree = ntree.lim, tunecontrol = tune.control(sampling = "cross", :  オブジェクト 'ntree.lim' がありません 


In [ ]:
summary(tune)

In [ ]:
plot(tune)

In [ ]:
fit <- randomForest(flag ~ ., data=df, mtry=3, ntree=500)

In [ ]:
fit$importance